# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os  # Module to interact with the operating system, such as accessing environment variables or file paths
import requests  # Library for making HTTP requests to access web pages or APIs
import json  # Provides functions to work with JSON data, such as loading and dumping JSON strings
from typing import (
    List,
)  # typing module provides standard way to specify types of variables and function parameters
# List object allows to specify the type of elements within a list
from dotenv import (
    load_dotenv,
)  # Loads environment variables from a `.env` file into the system's environment variables, useful for managing sensitive information like API keys
from bs4 import (
    BeautifulSoup,
)  # Imports BeautifulSoup from the bs4 library, which is used for parsing and extracting data from HTML and XML documents
from IPython.display import (
    Markdown,
    display,
    update_display,
)  # Imports Markdown, display, and update_display from the IPython.display module, allowing for the rendering of Markdown content and dynamically displaying or updating outputs in an IPython environment, such as Jupyter notebooks
from openai import (
    OpenAI,
)  # Imports OpenAI class to interact with the OpenAI API, allowing for generating responses using language models like GPT

In [2]:
# Initialize and constants

load_dotenv()  # Load environment variables from a .env file into the system's environment variables

# Retrieve the 'OPENAI_API_KEY' value from the environment variables
api_key = os.getenv("OPENAI_API_KEY")

# Check if the API key is valid by looking for specific patterns
if api_key and api_key.startswith("sk-proj-") and len(api_key) > 10:
    # If the API key looks correct, print a confirmation message
    print("API key looks good so far")
else:
    # If the API key seems incorrect, prompt the user to check for potential issues
    print(
        "There might be a problem with your API key? Please visit the troubleshooting notebook!"
    )

# Define the model that will be used for generating responses
MODEL = "gpt-4o-mini"

# Create an instance of OpenAI to interact with the OpenAI API
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage
class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        # object with 5 attributes & 1 method:
        # - 5 attributes:
        # 1. self.url: Stores the URL/address of the webpage.
        # 2. self.body: Stores the full raw HTML content.
        # 3. self.title: Extracts title from parsed HTML content if present.
        # 4. self.text: Extracts main visible text content from parsed HTML, excluding irrelevant tags like <script> or <style>.
        # 5. self.links: Extracts hyperlinks from parsed HTML.
        # - 1 method:
        # 1. get_contents(): Returns the title and main text content as a formatted string.

        # Store the URL of the website
        self.url = url
        # Send a GET request to fetch the webpage content
        response = requests.get(url)
        # Store the raw HTML content of the response
        self.body = response.content
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(self.body, "html.parser")
        # Extract the title of the webpage (if available), otherwise set it as "No title found"
        self.title = soup.title.string if soup.title else "No title found"

        # If the webpage has a <body> tag
        if soup.body:
            # Remove unnecessary tags such as <script>, <style>, <img>, and <input> from the body
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            # Extract and store the visible text content of the <body> with line breaks and trimmed whitespace
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            # If there's no <body> tag, set the text content as empty
            self.text = ""

        # Extract all <a> tags from the parsed HTML:
        # the <a> tags in HTML are anchor tags, which are used to define hyperlinks that link from one page to another;
        # the 'href' attribute within an <a> tag is what specifies the destination URL of the hyperlink,
        # for example: <a href="https://example.com">Visit Example</a>
        # In this case, href="https://example.com" is the link and .get('href') method extracts this URL value from each <a> tag
        # get() is used so that if <a> tag does not have a href, it will return 'None' rather than raise an error
        links = [link.get("href") for link in soup.find_all("a")]
        # Store only non-empty links (i.e., href attribute is present)
        self.links = [link for link in links if link]

    # Method to get the webpage title and contents in a formatted string
    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/08/06/outsmart/',
 'https://edwarddonner.com/2024/08/06/outsmart/',
 'https://edwarddonner.com/2024/06/26/choosing-the-right-llm-resources/

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  

It should:

    a) decide which links are relevant, and
    b) replace relevant links such as "/about" with "https://company.com/about".  
    
To illustrate this for the model, we will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/
https://edwarddonner

In [9]:
# Define a function named `get_links` that takes a URL as an argument
def get_links(url):
    # Create an instance of the Website class with the given URL, which will scrape the webpage
    website = Website(url)

    # Create a chat completion request using OpenAI's API:
    # openai is an instance of OpenAI Class that provides an interface to interact with OpenAI's API;
    # .chat attribute of openai object that allows to interact with with chat-based OpenAI models that
    #       can handle conversational input and produce conversational responses
    # .completions attribute of .chat attribute refers to the act of generating output text given an
    #       input prompt, the model's ability to predict the most suitable text to complete the given context
    # .create() method of .completions attribute sends a request to openai object to create a completion
    #       according to paramaters that are passed in:
    # .create(model = language model to use to generate completion, messages = list of prompts to provide
    #       context to conversation, etc)
    response = openai.chat.completions.create(
        # Specify the model to be used for generating the completion
        model=MODEL,
        # Provide the conversation messages for the model to work with
        messages=[
            # The system message to instruct the behavior of the model (e.g., providing context or guidelines)
            {"role": "system", "content": link_system_prompt},
            # The user message with the content generated based on the website data (
            # e.g., a question or prompt to interact with the website's links)
            {"role": "user", "content": get_links_user_prompt(website)},
        ],
        # Specify the format in which you want the response; in this case, requesting a JSON object
        response_format={"type": "json_object"},
    )

    # Extract the generated text from the response object returned from the OpenAI API call
    # .choices: attribute of the response object, a list that contains all the possible responses
    #          (or completions) that the model generated based on given input;
    # .choices[0]: from this list first (and in this case only) completion is selected.
    # .message: attribute of .choice, which contains a role (either "user", "assistant",
    #          or "system") to indicate who generated it and content (the generated response).
    #          a. User: Represents the person interacting with the AI, asking questions or giving commands;
    #          b. System: The context or persona for the assistant's behavior during the conversation.
    #          c. Assistant: Represents the AI itself, generating responses to fulfill the user's requests.
    # .content: attribute of .message, which contains the actual text content generated by the Assistant/model
    #          represented as a string representation in Python of a JSON object.
    result = response.choices[0].message.content

    # the return value is this result, but loads the string representation of the JSON object and converts it
    # into/parses it to a dictionary
    return json.loads(result)

In [10]:
anthropic = Website("https://anthropic.com")
anthropic.links

[]

In [11]:
get_links("https://anthropic.com")

{'links': [{'type': 'about page', 'url': 'https://anthropic.com/about'},
  {'type': 'careers page', 'url': 'https://anthropic.com/careers'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
# Define a function named 'get_all_details' that takes a URL as an argument
def get_all_details(url):
    # Initialize 'result' with the heading for the landing page details
    result = "Landing page:\n"

    # Create a Website object for the given URL, and append its content to 'result'
    result += Website(url).get_contents()

    # Get all links found on the webpage by using the 'get_links' function
    links = get_links(url)

    # Print out the found links to the console for debugging or information purposes
    print("Found links:", links)

    # Iterate through each link found in the "links" key of the returned dictionary
    for link in links["links"]:
        # Append the type of the link (e.g., internal or external) to 'result'
        result += f"\n\n{link['type']}\n"

        # Create a Website object for the link's URL and append its content to 'result'
        result += Website(link["url"]).get_contents()

    # Return the complete 'result' with all the details collected from the landing page and the links
    return result

In [13]:
print(get_all_details("https://anthropic.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/about'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}]}
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:




about page
Webpage Title:
Not Found \ Anthropic
Webpage Contents:




careers page
Webpage Title:
Careers \ Anthropic
Webpage Contents:





In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000]  # Truncate if more than 20,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/about'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}]}


'You are looking at a company called: Anthropic\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHome \\ Anthropic\nWebpage Contents:\n\n\n\n\nabout page\nWebpage Title:\nNot Found \\ Anthropic\nWebpage Contents:\n\n\n\n\ncareers page\nWebpage Title:\nCareers \\ Anthropic\nWebpage Contents:\n\n\n'

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)},
        ],
    )
    result = response.choices[0].message.content
    # Display the response content JSON string as formatted Markdown in the output cell
    # result: a JSON string that actually contains text formatted using Markdown syntax
    # markdown(): function that takes the JSON string stored in result and interprets it correctly as Markdown in Python
    # display(): function that takes the formatted Markdown content and displays it visually in the Jupyter Notebook cell
    # (if display were not called, the formatted Markdown content would be rendered as plain text)
    display(Markdown(result))

In [18]:
create_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/about'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}]}


# Anthropic Company Brochure

## Welcome to Anthropic

At Anthropic, we are on a mission to advance artificial intelligence in a way that is safe, ethical, and beneficial for everyone. Our commitment to harnessing the power of AI technology while prioritizing safety and humanity defines our approach and distinguishes us in the tech landscape.

## Company Culture

At Anthropic, our culture is rooted in collaboration and innovation. We believe that the best ideas come from diverse perspectives and teamwork, which helps us tackle the complex challenges associated with AI development. Our team is encouraged to think creatively, share freely, and work together toward common goals. We value transparency and encourage open discussions about our projects, methodologies, and ethical considerations.

Our workplace fosters professional growth and encourages continuous learning. We are committed to creating an inclusive environment where every employee feels valued and empowered to contribute. 

## Our Customers

Anthropic serves a wide range of clients and partners, focusing on those who share our vision of developing AI that is safe and beneficial. Although specific customer details were not provided, we strive to collaborate with organizations that prioritize ethical AI development and share our commitment to making a positive impact across various sectors.

## Careers at Anthropic

We are always on the lookout for passionate and talented individuals who are eager to help shape the future of AI. Joining Anthropic means becoming part of a mission-driven team focused on creating technology that aligns with human values.

### Why Work with Us?
- **Innovative Environment**: Work alongside leading experts in AI.
- **Commitment to Safety**: Be part of a team that prioritizes ethical considerations and safety in AI development.
- **Growth Opportunities**: We support professional development and continuous learning within our team.

Interested candidates can explore our careers page to discover current job openings and learn more about what it’s like to work at Anthropic.

---

For more information, please visit our [website](https://www.anthropic.com). Join us in redefining the future of AI!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
# Function to stream the creation of a brochure in real-time
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)},
        ],
        stream=True,  # This is the small adjustment that enables streaming of the result
        # instead of waiting for the entire response to be generated, streaming sends data 
        # in chunks as it becomes available, providing the typewriter animation effect
    )

    # Initialize an empty string to collect the response as it streams
    response = ""

    # Create a display handle with an empty Markdown output (i.e. without concrete content 
    # for now) for real-time updates (later):
    # display(): function that returns a display object (or handle)
    # Parameters define metadata of the display object:
    # Markdown(""): initializes an empty Markdown content (to be displayed later when
    #               content will be generated)
    # display_id=True: instructs the display() function to create a unique display ID
    #               once output is generated, allowing it to be referred to later
    #               This allows the display to be updated dynamically during streaming,
    #               ensuring real-time content updates in the same output cell
    display_handle = display(Markdown(""), display_id=True)

    # Iterate over each "chunk" of data that comes in from the stream
    for chunk in stream:
        # Add the streamed content (if available) to the ongoing response
        # chunk: a single incremental portion of the response, a dictionary including
        #        metadata (role of speaker, completion details, message deltas) and contents
        # .choices: an attribute of the chunk object, and it is a list containing the 
        #        available response choices
        # .choices[0]: since we are streaming, there is generally only one choice being
        #        returned at any given time, so the list will have one element, accessed by [0]
        # .delta: an attribute of the chosen element .choices[0] in the chunk object, 
        #        delta represents a small, incremental update or change
        # .content: an attribute of the delta object, it represents the actual text 
        #        content of the current chunk
        # or ' ': if content is not present (e.g., empty chunk), it defaults to '' to
        #        avoid errors
        response += chunk.choices[0].delta.content or ""

        # Remove unwanted formatting markers (e.g., ``` and "markdown") from the response to keep the display clean
        response = response.replace("```", "").replace("markdown", "")

        # Update the displayed Markdown content in real-time with the current response
        # update_display(): A function that updates an existing display with new content
        #          instead of creating a new display cell each time, ensuring continuous 
        #          output in the same cell
        # Markdown(response): Takes the response string, which is incrementally built during
        #          streaming, and converts it into a Markdown object for proper formatting
        # display_id=...: Identifies which existing display should be updated by linking it
        #         to a specific display that was created earlier
        # display_id=display_handle.display_id: Links the current update to the original 
        #         display created earlier, using its unique ID, ensuring that the correct 
        #         display is modified rather than creating a new one
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/about/'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers/'}]}


# Anthropic Company Brochure

---

## About Anthropic

Anthropic is an innovative technology company focused on advancing AI capabilities while prioritizing safety and ethical considerations. Although specific details from the "About" page are not available, Anthropic is recognized for its commitment to creating powerful AI systems that are aligned with human values, ensuring technology enhances society rather than disrupts it.

### Our Mission

At Anthropic, our mission is to conduct research and development in artificial intelligence that prioritizes safety, trustworthiness, and human alignment. We believe that as AI technologies become more integrated into everyday life, it is essential to keep human interests at the forefront of innovation.

---

## Company Culture

Anthropic fosters a dynamic and inclusive company culture that values diverse perspectives and collaborative work. We emphasize the importance of a positive work environment where employees can thrive both personally and professionally.

- **Inclusivity**: We celebrate diversity and strive to create an environment where every voice is heard.
- **Innovation**: Continuous learning and experimentation drive our approach to problem-solving and creative solutions.
- **Integrity**: We uphold high ethical standards in all our efforts, ensuring that our technological advancements are responsible and beneficial.

---

## Our Customers

Anthropic serves a wide range of clients, from startups to established enterprises, looking to harness the power of AI. Our customers appreciate our focus on developing reliable, safe, and effective AI systems that meet their specific needs and help them achieve their goals.

---

## Careers at Anthropic

We are always on the lookout for talented individuals to join our team. At Anthropic, we offer:

- **Challenging Projects**: Engage in meaningful work that shapes the future of AI and technology.
- **Professional Growth**: Access to learning opportunities, mentorship, and career development paths.
- **Competitive Benefits**: We provide a comprehensive compensation package designed to support our employees' well-being and work-life balance.

If you are passionate about AI and want to be part of a forward-thinking team, check our careers page for the latest job openings and opportunities to join us in our mission.

---

**Join us at Anthropic, where we lead the way to a new era of artificial intelligence!**

For more information, visit our website or contact us directly.

In [21]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'doc page', 'url': 'https://huggingface.co/docs'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}]}



# Welcome to Hugging Face! 🤗

## The AI Community Building the Future (And We Promise, It’s Not All Robots!)

At Hugging Face, we believe that every dreamer deserves access to cutting-edge machine learning tools. Our community is the friendly—and occasionally over-caffeinated—hub for AI enthusiasts, engineers, and beginners alike. Join us on this journey to transform the machine-learning universe, ***one relatable tweet at a time***!

---

## Our Offerings: More than Just Lots of Models!

- **400k+ Models & 100k+ Datasets**: Enough data to make your head spin faster than a neural network!  
- **Spaces**: Think of this as your AI playground, where your imagination can run wild without the fear of a messy room. 
- **HuggingChat**: Have a chat with our AI models—though we can’t guarantee they won't steal your favorite memes.

---

## Our Customers: Who’s in on the Fun?

More than **50,000 organizations** are using Hugging Face! That includes heavyweight champions like Google, Microsoft, and Amazon—basically, if there's an AI involved, we're probably in the mix! 🤓

---

## Company Culture: Join The Fam!

At Hugging Face, we embrace diversity, creativity, and just a smidge of quirkiness. Our team is comprised of **224 passionate individuals**, and we're always on the lookout for more spirited minds. Whether you’re a data scientist or a meme generator, there's a place for you here! 

- **Collaboration**: We prefer sharing ideas over coffee (or maybe an espresso shot or two). 
- **Open Source**: We’re all about that good karma, sharing our tools and tips with the community because we can’t store all this genius for ourselves! Your code has feelings too, you know? 

---

## Careers: Come Work With Us!

If you’re interested in blending fun with a future in AI, check out our **Current Openings**! Who knows? You might just land a gig where 'snack breaks' and 'model training' can occur simultaneously! 

The jobs are as diverse as our personality types. We don’t just hire engineers; we want artists, musicians, and poets too! 

---

## Pricing that’s as Transparent as Our Goals

- **Free Tier**: For those just dipping their toes into the ML water (don't worry, we promise it's not too cold!).
- **Pro at $9/month**: Unlock extras like high-fives from community members (virtually) and priority support!
- **Enterprise Hub**: Want your own data fortress with extra bells and whistles? It's just $20 per user, plus a layer of enterprise-grade security that could withstand a digital siege!

---

## Get Involved & Join Our Hugging Family! 🎉

Join us as we scale the peaks of machine learning (while avoiding 'deep learning' puns... or maybe not!). Visit our [website](https://huggingface.co) to sign up, collaborate, and contribute!

![We Hug, You Hug, We All Hug AI!](https://images.unsplash.com/photo-1644241440480-b8a40f7f456b)

*Because in the end, we're all just trying to make the world a little more 'hugging' and a lot less facepalmy!*

## Connect with Us
- [GitHub](https://github.com/huggingface) 
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://www.linkedin.com/company/huggingface)
- [Discord](https://discord.com/invite/huggingface)

*Get ready to join the AI revolution (and don't forget your sense of humor)!* 😊



<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>